In [ ]:
import datetime
import pandas as pd
import pyaurorax

aurorax = pyaurorax.PyAuroraX()

# AuroraX ML-enhanced searching

Integrated with the AuroraX search engine, there is the ability to search through ephemeris and conjunctions based on various machine learning model metadata that has been uploaded to the platform. 

Two use cases to highlight, but there are many more: 
  - finding all conjunctions between spacecrafts and the THEMIS ASIs where an ML model believes it is not cloudy in the THEMIS ASI
  - retrieve all 1-minute times where an ML model believes THEMIS ASIs have Amorphous Pulsating Aurora (APA) in the field of view.

More information about the available ML metadata can be found [here](https://docs.aurorax.space/ml/overview).

# Conjunction Searching

Let's first have a look at how to do conjunction searches, and find results by leveraging some ML-derived metadata.

## UCalgary Amorphous Pulsating Aurora (APA) model

UCalgary has developed a machine learning model for identifying Amorphous Pulsating Aurora (APA) in the THEMIS all-sky imagers. This is a binary classification performed on a 10-minute basis, and included in all THEMIS ASI AuroraX search engine 'ephemeris' records as a metadata field.

Metadata fields in AuroraX can be searched upon, allowing users to filter results based on them. Below, we're going to show an example of finding all 1-minute ephemeris records for any THEMIS ASI instrument where this particular model thinks there is APA in the field-of-view for that camera, over a 1 month period.

More information about this model can be found [here](https://docs.aurorax.space/ml/models/ucalgary_apa/).

In [ ]:
# set up search parameters
start = datetime.datetime(2008, 1, 1, 0, 0, 0)
end = datetime.datetime(2008, 1, 31, 23, 59, 59)
ground_params = [{
    "programs": ["themis-asi"],
    "ephemeris_metadata_filters": {
        "logical_operator": "AND",
        "expressions": [
            {
                "key": "calgary_apa_ml_v1",
                "operator": "in",
                "values": ["classified as APA"]  # only find records that were classified as APA
            },
            {
                "key": "calgary_apa_ml_v1_confidence",
                "operator": ">=",
                "values": ["95"]  # with a confidence of at least 95%
            },
        ]
    }
}]
space_params = [{
    "programs": ["themis"],
    "hemisphere": ["northern"],
}]
conjunction_types = [pyaurorax.search.CONJUNCTION_TYPE_NBTRACE]
distance = 500

# perform the search
s = aurorax.search.conjunctions.search(
    start=start,
    end=end,
    distance=distance,
    ground=ground_params,
    space=space_params,
    conjunction_types=conjunction_types,
    verbose=True,
)

[2024-12-13 20:20:30.741796] Search object created
[2024-12-13 20:20:32.805063] Request submitted
[2024-12-13 20:20:32.805300] Request ID: ce059fcd-5db1-4be7-8947-39094b8deaf2
[2024-12-13 20:20:32.805335] Request details available at: https://api.aurorax.space/api/v1/conjunctions/requests/ce059fcd-5db1-4be7-8947-39094b8deaf2
[2024-12-13 20:20:32.805356] Waiting for data ...
[2024-12-13 20:20:35.938072] Data is now available
[2024-12-13 20:20:35.938470] Retrieving data ...
[2024-12-13 20:20:38.728521] Retrieved 52.8 kB of data containing 8 records


In [4]:
# show the conjunction results
#
# NOTE: while here we format the results into a Pandas dataframe, this
# is not required. We actually don't include Pandas as a dependency since
# it's used simply as a nice add-on to view data. If you're good with slicing
# and dicing lists and dictionaries, you'll be fine without it.
conjunctions = [d.__dict__ for d in s.data]
df = pd.DataFrame(conjunctions)
df.sort_values("start")

,conjunction_type,start,end,data_sources,min_distance,max_distance,events,closest_epoch,farthest_epoch
0,nbtrace,2008-01-01T11:30:00,2008-01-01T11:36:00,"[DataSource(identifier=45, program='themis-asi...",383.962128,428.502480,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2008-01-01T11:30:00,2008-01-01T11:36:00
1,nbtrace,2008-01-01T11:30:00,2008-01-01T11:59:00,"[DataSource(identifier=45, program='themis-asi...",340.029413,499.355723,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2008-01-01T11:30:00,2008-01-01T11:59:00
6,nbtrace,2008-01-08T05:50:00,2008-01-08T05:52:00,"[DataSource(identifier=49, program='themis-asi...",485.213758,499.938201,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2008-01-08T05:50:00,2008-01-08T05:52:00
3,nbtrace,2008-01-14T04:50:00,2008-01-14T04:59:00,"[DataSource(identifier=49, program='themis-asi...",150.330448,200.986916,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2008-01-14T04:50:00,2008-01-14T04:59:00
5,nbtrace,2008-01-14T04:50:00,2008-01-14T04:59:00,"[DataSource(identifier=49, program='themis-asi...",203.965929,271.159251,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2008-01-14T04:50:00,2008-01-14T04:59:00
4,nbtrace,2008-01-19T08:02:00,2008-01-19T08:19:00,"[DataSource(identifier=51, program='themis-asi...",318.807754,390.216280,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2008-01-19T08:05:00,2008-01-19T08:19:00
2,nbtrace,2008-01-19T08:04:00,2008-01-19T08:09:00,"[DataSource(identifier=46, program='themis-asi...",393.493000,494.091856,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2008-01-19T08:09:00,2008-01-19T08:04:00
7,nbtrace,2008-01-25T06:40:00,2008-01-25T06:49:00,"[DataSource(identifier=54, program='themis-asi...",327.930714,372.530865,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2008-01-25T06:40:00,2008-01-25T06:49:00


In [5]:
# we can explore these results in Swarm-Aurora too if we want
#
# open browser to Swarm-Aurora for this search
# aurorax.search.conjunctions.swarmaurora.open_in_browser(s)
# aurorax.search.conjunctions.swarmaurora.open_in_browser(s, browser='google-chrome')

# alternatively, you can also just get the web URL for Swarm-Aurora for this
# specific request, and copy paste it into the browser manually.
print(aurorax.search.conjunctions.swarmaurora.get_url(s))

https://swarm-aurora.com/conjunctionFinder?aurorax_request_id=ce059fcd-5db1-4be7-8947-39094b8deaf2


## UCalgary cloud model

UCalgary has developed a machine learning model for identifying cloud in the THEMIS all-sky imagers. This is a binary classification performed on a 10-minute basis, and included in all THEMIS ASI AuroraX search engine 'ephemeris' records as a metadata field.

Below, we're going to show an example of finding all 1-minute ephemeris records for any THEMIS ASI instrument where this particular model thinks there is cloud in the field-of-view at Gillam, over a 7 day period.

More information about this model can be found [here](https://docs.aurorax.space/ml/models/ucalgary_cloud/).

In [6]:
# set up search parameters
start = datetime.datetime(2020, 1, 1, 0, 0, 0)
end = datetime.datetime(2020, 1, 15, 23, 59, 59)
ground_params = [{
    "programs": ["themis-asi"],
    "ephemeris_metadata_filters": {
        "logical_operator": "AND",
        "expressions": [
            {
                "key": "calgary_cloud_ml_v1",
                "operator": "in",
                "values": ["classified as not cloudy"]  # only find records that were classified as not cloud
            },
            {
                "key": "calgary_cloud_ml_v1_confidence",
                "operator": ">=",
                "values": ["75"]  # with a confidence of at least 75%
            },
        ]
    }
}]
space_params = [{
    "programs": ["swarm", "elfin"],
    "hemisphere": ["northern"],
}]
conjunction_types = [pyaurorax.search.CONJUNCTION_TYPE_NBTRACE]
distance = 500

# perform the search
s = aurorax.search.conjunctions.search(
    start=start,
    end=end,
    distance=distance,
    ground=ground_params,
    space=space_params,
    conjunction_types=conjunction_types,
    verbose=True,
)

[2024-12-13 20:32:42.574591] Search object created
[2024-12-13 20:32:45.076212] Request submitted
[2024-12-13 20:32:45.076368] Request ID: 0f9a9f34-7018-42df-81f7-538c1056c3da
[2024-12-13 20:32:45.076382] Request details available at: https://api.aurorax.space/api/v1/conjunctions/requests/0f9a9f34-7018-42df-81f7-538c1056c3da
[2024-12-13 20:32:45.076390] Waiting for data ...
[2024-12-13 20:32:47.635497] Data is now available
[2024-12-13 20:32:47.635625] Retrieving data ...
[2024-12-13 20:32:50.547224] Retrieved 2.2 MB of data containing 270 records


In [9]:
# show the first 10 conjunction results
conjunctions = [d.__dict__ for d in s.data]
df = pd.DataFrame(conjunctions)
df.sort_values("start")[0:10]

,conjunction_type,start,end,data_sources,min_distance,max_distance,events,closest_epoch,farthest_epoch
48,nbtrace,2020-01-01T00:08:00,2020-01-01T00:09:00,"[DataSource(identifier=53, program='themis-asi...",192.029390,336.631805,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:09:00,2020-01-01T00:08:00
155,nbtrace,2020-01-01T00:08:00,2020-01-01T00:09:00,"[DataSource(identifier=53, program='themis-asi...",206.588207,260.819127,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:09:00,2020-01-01T00:08:00
86,nbtrace,2020-01-01T01:45:00,2020-01-01T01:46:00,"[DataSource(identifier=44, program='themis-asi...",205.885002,420.474697,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T01:46:00,2020-01-01T01:45:00
6,nbtrace,2020-01-01T01:46:00,2020-01-01T01:47:00,"[DataSource(identifier=44, program='themis-asi...",284.420850,493.197476,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T01:46:00,2020-01-01T01:47:00
27,nbtrace,2020-01-01T03:15:00,2020-01-01T03:16:00,"[DataSource(identifier=47, program='themis-asi...",412.812856,489.362899,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T03:16:00,2020-01-01T03:15:00
120,nbtrace,2020-01-01T03:15:00,2020-01-01T03:16:00,"[DataSource(identifier=47, program='themis-asi...",385.355069,414.231784,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T03:16:00,2020-01-01T03:15:00
256,nbtrace,2020-01-01T04:07:00,2020-01-01T04:08:00,"[DataSource(identifier=53, program='themis-asi...",343.134412,396.842659,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T04:07:00,2020-01-01T04:08:00
185,nbtrace,2020-01-01T04:49:00,2020-01-01T04:50:00,"[DataSource(identifier=65, program='themis-asi...",244.886114,355.591219,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T04:49:00,2020-01-01T04:50:00
68,nbtrace,2020-01-01T04:49:00,2020-01-01T04:50:00,"[DataSource(identifier=65, program='themis-asi...",323.719101,352.655457,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T04:49:00,2020-01-01T04:50:00
219,nbtrace,2020-01-01T05:18:00,2020-01-01T05:19:00,"[DataSource(identifier=53, program='themis-asi...",371.020465,478.895130,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T05:19:00,2020-01-01T05:18:00


In [10]:
# have a look at the results in Swarm-Aurora
print(aurorax.search.conjunctions.swarmaurora.get_url(s))

https://swarm-aurora.com/conjunctionFinder?aurorax_request_id=0f9a9f34-7018-42df-81f7-538c1056c3da


## Oslo Aurora THEMIS (OATH) model

The OATH model was developed by [Clausen & Nickisch](https://doi.org/10.1029/2018JA025274), and has been run by the AuroraX team for all THEMIS ASI data in the AuroraX search engine. has developed a machine learning model for identifying cloud in the THEMIS all-sky imagers.

- Paper: https://doi.org/10.1029/2018JA025274
- More information: http://tid.uio.no/plasma/oath

This paper has two versions of the model, one that is a binary classifier (aurora/no aurora), and one that is a 6-class classifier. AuroraX currently only contains metadata derived from the 6-class model.

The OATH mode is a binary classification performed on a 10-minute basis, and included in all THEMIS ASI AuroraX search engine 'ephemeris' records as a metadata field.

Below, we're going to show an example of finding all 1-minute ephemeris records for any THEMIS ASI instrument where the OATH model has classified the minute as diffuse or discrete aurora, over a 1 day period.

More information about the AuroraX utilization of this model can be found [here](https://docs.aurorax.space/ml/models/clausen_oath/).

In [11]:
# set up search parameters
start = datetime.datetime(2015, 1, 1, 0, 0, 0)
end = datetime.datetime(2015, 1, 31, 23, 59, 59)
ground_params = [{
    "programs": ["themis-asi"],
    "ephemeris_metadata_filters": {
        "logical_operator": "AND",
        "expressions": [{
            "key": "clausen_ml_oath",
            "operator": "in",
            "values": [  # only find records that have one of these three classifications
                "classified as diffuse", "classified as discrete", "classified as arc"
            ]
        }]
    }
}]
space_params = [{
    "programs": ["dmsp"],
    "hemisphere": ["northern"],
}]
conjunction_types = [pyaurorax.search.CONJUNCTION_TYPE_NBTRACE]
distance = 500

# perform the search
s = aurorax.search.conjunctions.search(
    start=start,
    end=end,
    distance=distance,
    ground=ground_params,
    space=space_params,
    conjunction_types=conjunction_types,
    verbose=True,
)

[2024-12-13 20:39:19.510449] Search object created
[2024-12-13 20:39:21.981418] Request submitted
[2024-12-13 20:39:21.981545] Request ID: ba825ec7-9976-4d42-8c54-fd979fd4c52a
[2024-12-13 20:39:21.981559] Request details available at: https://api.aurorax.space/api/v1/conjunctions/requests/ba825ec7-9976-4d42-8c54-fd979fd4c52a
[2024-12-13 20:39:21.981567] Waiting for data ...
[2024-12-13 20:39:24.793019] Data is now available
[2024-12-13 20:39:24.793154] Retrieving data ...
[2024-12-13 20:39:27.936641] Retrieved 1.5 MB of data containing 222 records


In [12]:
# show the first 10 conjunction results
conjunctions = [d.__dict__ for d in s.data]
df = pd.DataFrame(conjunctions)
df.sort_values("start")[0:10]

,conjunction_type,start,end,data_sources,min_distance,max_distance,events,closest_epoch,farthest_epoch
180,nbtrace,2015-01-01T06:38:00,2015-01-01T06:40:00,"[DataSource(identifier=71, program='themis-asi...",129.676030,477.204038,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2015-01-01T06:39:00,2015-01-01T06:38:00
101,nbtrace,2015-01-01T10:02:00,2015-01-01T10:02:00,"[DataSource(identifier=51, program='themis-asi...",38.161652,38.161652,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2015-01-01T10:02:00,2015-01-01T10:02:00
86,nbtrace,2015-01-01T10:58:00,2015-01-01T10:58:00,"[DataSource(identifier=48, program='themis-asi...",424.850074,424.850074,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2015-01-01T10:58:00,2015-01-01T10:58:00
143,nbtrace,2015-01-01T13:36:00,2015-01-01T13:36:00,"[DataSource(identifier=53, program='themis-asi...",477.845592,477.845592,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2015-01-01T13:36:00,2015-01-01T13:36:00
67,nbtrace,2015-01-01T14:18:00,2015-01-01T14:19:00,"[DataSource(identifier=45, program='themis-asi...",429.648698,469.288619,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2015-01-01T14:19:00,2015-01-01T14:18:00
148,nbtrace,2015-01-01T16:00:00,2015-01-01T16:00:00,"[DataSource(identifier=57, program='themis-asi...",352.260006,352.260006,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2015-01-01T16:00:00,2015-01-01T16:00:00
216,nbtrace,2015-01-01T20:10:00,2015-01-01T20:11:00,"[DataSource(identifier=71, program='themis-asi...",285.473795,299.878087,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2015-01-01T20:10:00,2015-01-01T20:11:00
40,nbtrace,2015-01-01T21:31:00,2015-01-01T21:33:00,"[DataSource(identifier=71, program='themis-asi...",75.318471,425.663394,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2015-01-01T21:32:00,2015-01-01T21:33:00
181,nbtrace,2015-01-02T06:23:00,2015-01-02T06:24:00,"[DataSource(identifier=71, program='themis-asi...",335.819298,365.536067,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2015-01-02T06:23:00,2015-01-02T06:24:00
201,nbtrace,2015-01-02T09:00:00,2015-01-02T09:00:00,"[DataSource(identifier=71, program='themis-asi...",356.122232,356.122232,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2015-01-02T09:00:00,2015-01-02T09:00:00


In [13]:
# have a look at the results in Swarm-Aurora
print(aurorax.search.conjunctions.swarmaurora.get_url(s))

https://swarm-aurora.com/conjunctionFinder?aurorax_request_id=ba825ec7-9976-4d42-8c54-fd979fd4c52a


# Ephemeris searching

Let's next have a look at doing ephemeris searches and leverage the ML-derived metadata.

## UCalgary Amorphous Pulsating Aurora (APA) model

In [14]:
# set timeframe and program values
start = datetime.datetime(2008, 1, 1, 0, 0, 0)
end = datetime.datetime(2008, 1, 31, 23, 59, 59)
programs = ["themis-asi"]

# to filter based the ML model data, we need to perform the ephemeris
# search using metadata filters.
#
# below, we're going to filter to only retrieve ephemeris data for when
# the 'calgary_apa_ml_v1' field says 'classified as APA', and has a
# confidence score greater than or equal to 95%.
metadata_filters_logical_operator = "AND"
metadata_filters = [
    {
        "key": "calgary_apa_ml_v1",
        "operator": "in",
        "values": [
            "classified as APA"  # only find records that were classified as APA
        ]
    },
    {
        "key": "calgary_apa_ml_v1_confidence",
        "operator": ">=",
        "values": [
            "95"  # with a confidence of at least 95%
        ]
    }
]

# perform the search
s = aurorax.search.ephemeris.search(start=start,
                                    end=end,
                                    programs=programs,
                                    metadata_filters_logical_operator=metadata_filters_logical_operator,
                                    metadata_filters=metadata_filters,
                                    verbose=True)

[2024-12-13 20:40:19.926507] Search object created
[2024-12-13 20:40:25.113598] Request submitted
[2024-12-13 20:40:25.113792] Request ID: 6696ef27-5b3f-4d7b-9d3a-e6e4c4a236a2
[2024-12-13 20:40:25.113820] Request details available at: https://api.aurorax.space/api/v1/ephemeris/requests/6696ef27-5b3f-4d7b-9d3a-e6e4c4a236a2
[2024-12-13 20:40:25.113835] Waiting for data ...
[2024-12-13 20:40:32.117443] Data is now available
[2024-12-13 20:40:32.117817] Retrieving data ...
[2024-12-13 20:40:36.146732] Retrieved 18.7 MB of data containing 5407 records


In [17]:
# show the first 10 conjunction results
conjunctions = [d.__dict__ for d in s.data]
df = pd.DataFrame(conjunctions)
df.sort_values("epoch")[0:10]

,data_source,epoch,location_geo,nbtrace,sbtrace,location_gsm,metadata
5117,"DataSource(identifier=65, program='themis-asi'...",2008-01-01 05:50:00,"Location(lat=66.971312, lon=-160.437679)","Location(lat=66.971312, lon=-160.437679)","Location(lat=-56.08080707101569, lon=162.16676...","Location(lat=None, lon=None)","{'clausen_ml_oath': 'classified as no aurora',..."
5118,"DataSource(identifier=65, program='themis-asi'...",2008-01-01 05:51:00,"Location(lat=66.971312, lon=-160.437679)","Location(lat=66.971312, lon=-160.437679)","Location(lat=-56.08080717793904, lon=162.16676...","Location(lat=None, lon=None)","{'clausen_ml_oath': 'classified as arc', 'calg..."
5119,"DataSource(identifier=65, program='themis-asi'...",2008-01-01 05:52:00,"Location(lat=66.971312, lon=-160.437679)","Location(lat=66.971312, lon=-160.437679)","Location(lat=-56.08080728486236, lon=162.16676...","Location(lat=None, lon=None)","{'clausen_ml_oath': 'classified as discrete', ..."
5120,"DataSource(identifier=65, program='themis-asi'...",2008-01-01 05:53:00,"Location(lat=66.971312, lon=-160.437679)","Location(lat=66.971312, lon=-160.437679)","Location(lat=-56.08080739178574, lon=162.16676...","Location(lat=None, lon=None)","{'clausen_ml_oath': 'classified as discrete', ..."
5121,"DataSource(identifier=65, program='themis-asi'...",2008-01-01 05:54:00,"Location(lat=66.971312, lon=-160.437679)","Location(lat=66.971312, lon=-160.437679)","Location(lat=-56.080807498709085, lon=162.1667...","Location(lat=None, lon=None)","{'clausen_ml_oath': 'classified as arc', 'calg..."
5122,"DataSource(identifier=65, program='themis-asi'...",2008-01-01 05:55:00,"Location(lat=66.971312, lon=-160.437679)","Location(lat=66.971312, lon=-160.437679)","Location(lat=-56.080807605632415, lon=162.1667...","Location(lat=None, lon=None)","{'clausen_ml_oath': 'classified as moon', 'cal..."
5123,"DataSource(identifier=65, program='themis-asi'...",2008-01-01 05:56:00,"Location(lat=66.971312, lon=-160.437679)","Location(lat=66.971312, lon=-160.437679)","Location(lat=-56.08080771255578, lon=162.16676...","Location(lat=None, lon=None)","{'clausen_ml_oath': 'classified as arc', 'calg..."
5124,"DataSource(identifier=65, program='themis-asi'...",2008-01-01 05:57:00,"Location(lat=66.971312, lon=-160.437679)","Location(lat=66.971312, lon=-160.437679)","Location(lat=-56.080807819479084, lon=162.1667...","Location(lat=None, lon=None)","{'clausen_ml_oath': 'classified as arc', 'calg..."
5125,"DataSource(identifier=65, program='themis-asi'...",2008-01-01 05:58:00,"Location(lat=66.971312, lon=-160.437679)","Location(lat=66.971312, lon=-160.437679)","Location(lat=-56.08080792640243, lon=162.16676...","Location(lat=None, lon=None)","{'clausen_ml_oath': 'classified as arc', 'calg..."
5126,"DataSource(identifier=65, program='themis-asi'...",2008-01-01 05:59:00,"Location(lat=66.971312, lon=-160.437679)","Location(lat=66.971312, lon=-160.437679)","Location(lat=-56.08080803332583, lon=162.16677...","Location(lat=None, lon=None)","{'clausen_ml_oath': 'classified as arc', 'calg..."


In [ ]:
# print the results of the first 10 ephemeris records as a table showing just
# the classification and confidence score
#
# NOTE: you can use Pandas here like earlier, but here's an example of looking
# at the results without it
print(f"{'Timestamp':<{25}}"
      f"{'calgary_apa_ml_v1':<{25}} "
      f"{'confidence':<{18}}")
print("====================================================================")
for i in range(10):
    timestamp_str = s.data[i].epoch.strftime('%Y-%m-%d %H:%M:%S')
    print(f"{timestamp_str:<{25}}"
          f"{s.data[i].metadata['calgary_apa_ml_v1']:<{23}} "
          f"  {s.data[i].metadata['calgary_apa_ml_v1_confidence']:<{18}} ")

Timestamp                calgary_apa_ml_v1         confidence        
2008-01-01 11:30:00      classified as APA         99.96              
2008-01-01 11:31:00      classified as APA         99.96              
2008-01-01 11:32:00      classified as APA         99.96              
2008-01-01 11:33:00      classified as APA         99.96              
2008-01-01 11:34:00      classified as APA         99.96              
2008-01-01 11:35:00      classified as APA         99.96              
2008-01-01 11:36:00      classified as APA         99.96              
2008-01-01 11:37:00      classified as APA         99.96              
2008-01-01 11:38:00      classified as APA         99.96              
2008-01-01 11:39:00      classified as APA         99.96              


## UCalgary cloud model

In [ ]:
# set up search parameters
start = datetime.datetime(2008, 1, 1, 0, 0, 0)
end = datetime.datetime(2008, 1, 7, 23, 59, 59)
programs = ["themis-asi"]
platforms = ["gillam"]

# we're going to filter to only retrieve ephemeris data for when
# the 'calgary_cloud_ml_v1' field says 'classified as not cloudy',
# and has a confidence score greater than or equal to 75%.
metadata_filters_logical_operator = "AND"
metadata_filters = [{
    "key": "calgary_cloud_ml_v1",
    "operator": "in",
    "values": ["classified as not cloudy"]
}, {
    "key": "calgary_cloud_ml_v1_confidence",
    "operator": ">",
    "values": ["75"]
}]

# Perform the search
s = aurorax.search.ephemeris.search(start=start,
                                    end=end,
                                    programs=programs,
                                    platforms=platforms,
                                    metadata_filters_logical_operator=metadata_filters_logical_operator,
                                    metadata_filters=metadata_filters,
                                    verbose=True)

[2024-11-27 22:15:44.030167] Search object created
[2024-11-27 22:15:44.059359] Request submitted
[2024-11-27 22:15:44.059426] Request ID: 417610af-5ebf-40cb-a3f9-bdbeecfe96c2
[2024-11-27 22:15:44.059453] Request details available at: https://api.aurorax.space/api/v1/ephemeris/requests/417610af-5ebf-40cb-a3f9-bdbeecfe96c2
[2024-11-27 22:15:44.059476] Waiting for data ...
[2024-11-27 22:15:45.530094] Checking for data ...
[2024-11-27 22:15:45.953475] Data is now available
[2024-11-27 22:15:45.953686] Retrieving data ...
[2024-11-27 22:15:46.246634] Retrieved 2.1 MB of data containing 597 records


In [17]:
# Again, print some results
print(f"{'Timestamp':<{25}}"
      f"{'Classification':<{30}} "
      f"{'Confidence':<{18}}")
print("====================================================================")
for i in range(10):
    timestamp_str = s.data[i].epoch.strftime('%Y-%m-%d %H:%M:%S')
    print(f"{timestamp_str:<{25}}"
          f"{s.data[i].metadata['calgary_cloud_ml_v1']:<{28}} "
          f"  {s.data[i].metadata['calgary_cloud_ml_v1_confidence']:<{18}} ")

Timestamp                Classification                 Confidence        
2008-01-01 01:11:00      classified as cloudy           79.45              
2008-01-01 01:19:00      classified as cloudy           79.45              
2008-01-01 01:20:00      classified as not cloudy       73.82              
2008-01-01 01:22:00      classified as not cloudy       73.82              
2008-01-01 01:25:00      classified as not cloudy       73.82              
2008-01-01 01:26:00      classified as not cloudy       73.82              
2008-01-01 01:27:00      classified as not cloudy       73.82              
2008-01-01 01:28:00      classified as not cloudy       73.82              
2008-01-01 01:29:00      classified as not cloudy       73.82              
2008-01-01 01:30:00      classified as not cloudy       87.03              


## Oslo Aurora THEMIS (OATH) model

In [ ]:
# set up search parameters
start = datetime.datetime(2008, 1, 1, 0, 0, 0)
end = datetime.datetime(2008, 1, 1, 23, 59, 59)
programs = ["themis-asi"]

# we're going to filter to only retrieve ephemeris data for when
# the 'clausen_ml_oath' field says 'classified as diffuse' or
# 'classified as discrete'
metadata_filters_logical_operator = "AND"
metadata_filters = [{
    "key": "clausen_ml_oath",
    "operator": "in",
    "values": [
        "classified as diffuse",
        "classified as discrete",
    ]
}]

# Perform the search
s = aurorax.search.ephemeris.search(start=start,
                                    end=end,
                                    programs=programs,
                                    metadata_filters_logical_operator=metadata_filters_logical_operator,
                                    metadata_filters=metadata_filters,
                                    verbose=True)

[2024-11-27 22:30:29.991861] Search object created
[2024-11-27 22:30:30.016479] Request submitted
[2024-11-27 22:30:30.016520] Request ID: 5c66820d-32c4-49e1-860c-4853cb799188
[2024-11-27 22:30:30.016532] Request details available at: https://api.aurorax.space/api/v1/ephemeris/requests/5c66820d-32c4-49e1-860c-4853cb799188
[2024-11-27 22:30:30.016566] Waiting for data ...
[2024-11-27 22:30:31.444297] Checking for data ...
[2024-11-27 22:30:31.863775] Data is now available
[2024-11-27 22:30:31.863949] Retrieving data ...
[2024-11-27 22:30:32.447506] Retrieved 5.3 MB of data containing 1532 records


In [14]:
# Again, print some results
print(f"{'Timestamp':<{25}}"
      f"{'Classification':<{18}}")
print("=================================================")

for i in range(10):
    timestamp_str = s.data[i].epoch.strftime('%Y-%m-%d %H:%M:%S')
    print(f"{timestamp_str:<{23}}"
          f"  {s.data[i].metadata['clausen_ml_oath']:<{18}} ")

Timestamp                Classification    
2008-01-01 01:11:00      classified as discrete 
2008-01-01 01:19:00      classified as diffuse 
2008-01-01 01:20:00      classified as diffuse 
2008-01-01 01:22:00      classified as discrete 
2008-01-01 01:25:00      classified as diffuse 
2008-01-01 01:26:00      classified as diffuse 
2008-01-01 01:27:00      classified as diffuse 
2008-01-01 01:28:00      classified as diffuse 
2008-01-01 01:29:00      classified as diffuse 
2008-01-01 01:30:00      classified as diffuse 
